## Task

In this task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [141]:
# Import necessary libraries
import pandas as pd 
from datetime import date 
import numpy as np 
from fuzzywuzzy import process 
import chardet  # For character encoding detection (if needed)

# Set seed for reproducibility
np.random.seed(0) 

# Load store income data from CSV file
# Note: Replace 'store_income_data_task.csv' with the actual file path
data_income = pd.read_csv("store_income_data_task.csv") 

# Display the first few rows of the DataFrame
data_income.head()

,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States/
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,Britain
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,Britain/
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

Assuming you have a DataFrame named 'data_income' with a 'country' column

In [208]:
# Get unique countries from the DataFrame
unique_countries = data_income['country'].unique()

# Print the number of unique countries
print(f"There are {len(unique_countries)} unique countries")

# Print the list of unique countries
print(unique_countries)

There are 4 unique countries
['United States' 'United Kingdom' 'South Africa' nan]


In [145]:
# Convert to lower case
data_income['country'] = data_income['country'].str.lower()

# Remove trailing white spaces
data_income['country'] = data_income['country'].str.strip()

# Let us view the data
countries = data_income['country'].unique()
print(f"There are {len(countries)} unique countries")
countries

There are 37 unique countries


array(['united states/', 'britain', 'united states', 'britain/',
       'united kingdom', 'u.k.', 'sa', 'u.k/', 'america', nan, 's.a.',
       'england', 'uk', 's.a./', 'u.k', 'america/', 'sa.', '', 'uk.',
       'england/', 'united states of america', 'uk/', 'sa/', 'england.',
       'america.', 's.a..', 'united states of america.',
       'united states of america/', 'united states.',
       's. africasouth africa', 'britain.', '/', 'united kingdom.',
       's. africasouth africa/', 'united kingdom/',
       's. africasouth africa.', '.'], dtype=object)

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [147]:
# Get the top 10 closest matches to "united kingdom"
matches = fuzzywuzzy.process.extract("uk", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# Print the matches
print(matches)

[('uk', 100), ('uk.', 100), ('uk/', 100), ('u.k.', 40), ('u.k/', 40), ('u.k', 40), ('united states/', 13), ('united states', 13), ('united states.', 13), ('united kingdom', 12)]


In [149]:
# Function to replace rows in the provided column of the provided DataFrame
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = data_income[column].unique()

    # Get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings,
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = data_income[column].isin(close_matches)

    # Replace all rows with close matches with the input matches
    data_income.loc[rows_with_matches, column] = string_to_match

    # Let us know when the function is done
    print("All done!")

In [151]:
replace_matches_in_column(df=data_income, column='country', string_to_match="united kingdom")
replace_matches_in_column(df=data_income, column='country', string_to_match="united states")
replace_matches_in_column(df=data_income, column='country', string_to_match="united states of america")
replace_matches_in_column(df=data_income, column='country', string_to_match="south africa")
replace_matches_in_column(df=data_income, column='country', string_to_match="sa")
replace_matches_in_column(df=data_income, column='country', string_to_match="uk")
replace_matches_in_column(df=data_income, column='country', string_to_match="england")
replace_matches_in_column(df=data_income, column='country', string_to_match="britain")

All done!
All done!
All done!
All done!
All done!
All done!
All done!
All done!


In [153]:
# Let us view the data
countries = data_income['country'].unique()
print(f"There are {len(countries)} unique countries")
countries

There are 23 unique countries


array(['united states', 'britain', 'united kingdom', 'u.k.', 'sa', 'u.k/',
       'america', nan, 's.a.', 'england', 'uk', 's.a./', 'u.k',
       'america/', '', 'united states of america', 'america.', 's.a..',
       's. africasouth africa', '/', 's. africasouth africa/',
       's. africasouth africa.', '.'], dtype=object)

In [155]:
data_income['country'] = data_income['country'].replace(['britain', 'united kingdom', 'u.k.', 'u.k/', 'england', 'uk', 'u.k'],'United Kingdom')
data_income['country'] = data_income['country'].replace(['united states', 'america', 'united states of america', 'america.', 'america/'],'United States')
data_income['country'] = data_income['country'].replace(['sa', 's.a.', 's.a./', 's.a..', 's. africasouth africa', 's. africasouth africa/', 's. africasouth africa.'],'South Africa')

In [157]:
# Let us view the data
countries = data_income['country'].unique()
print(f"There are {len(countries)} unique countries")
countries

There are 7 unique countries


array(['United States', 'United Kingdom', 'South Africa', nan, '', '/',
       '.'], dtype=object)

In [159]:
# Replacing '', '/', '.' to nan
data_income['country'] = data_income['country'].replace(['', '/', '.'], np.nan)

In [161]:
# get all the unique values in the 'country' column
countries = data_income['country'].unique()

print(f"There are {len(countries)} unique countries")
countries

There are 4 unique countries


array(['United States', 'United Kingdom', 'South Africa', nan],
      dtype=object)

3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [171]:
data_income.head()

,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,United Kingdom
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,United Kingdom
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom


In [165]:
data_income.dtypes

id                int64
store_name       object
store_email      object
department       object
income           object
date_measured    object
country          object
dtype: object

In [200]:
# Create a new column, date_parsed, with the parsed dates
data_income['date_parsed'] = pd.to_datetime(data_income['date_measured'], format='%d-%m-%Y')

In [202]:
data_income.head()

,id,store_name,store_email,department,income,date_measured,country,date_parsed,days_ago
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States,2006-02-04,6929
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,United Kingdom,2006-01-04,6960
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States,2003-09-12,7805
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,United Kingdom,2006-05-08,6836
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom,1973-01-21,18996


In [210]:
# Get today's date
today = pd.to_datetime(today)

# Create a new column named 'days_ago' with the difference between today and 'date_measured'
data_income['days_ago'] = (today - data_income['date_parsed']).dt.days

data_income.head()

,id,store_name,store_email,department,income,date_measured,country,date_parsed,days_ago
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States,2006-02-04,6929
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,United Kingdom,2006-01-04,6960
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States,2003-09-12,7805
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,United Kingdom,2006-05-08,6836
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom,1973-01-21,18996
